# Process Deforestation Data from Hansen et al.

In this notebook we will process the tree cover and deforestation data from Hansen et al. This map data is divided into 504 granules of 10x10 degrees, spanning from 60 degrees South to 80 degrees North

In [4]:
import numpy as np
import rasterio
import PIL
from PIL import Image
from pathlib import Path
import pickle
import json
import multiprocessing

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
Image.MAX_IMAGE_PIXELS = 2000000000  # Increase PIL image load limit

In [13]:
longitude_range = (-180, 180)
latitude_range = (-60, 80)

pixels_per_degree = 3

longitude_size = (longitude_range[1] - longitude_range[0]) * pixels_per_degree
latitude_size = (latitude_range[1] - latitude_range[0]) * pixels_per_degree
granule_size = longitude_size / 36

km_to_px_EC = 2 * np.pi * 6371 / 360 / pixels_per_degree

img_common_size = (latitude_size, longitude_size)
print(img_common_size)
print(f"Each pixel will have a side of {km_to_px_EC:.3f} km at the ecuator.")
print(f"Each granule will have {granule_size:.0f} pixels to a side.")

(420, 1080)
Each pixel will have a side of 37.065 km at the ecuator.
Each granule will have 30 pixels to a side.


In [18]:
def granule_from_tif(tif_location, granule_size):
    with rasterio.open(tif_location) as dataset_reader:
        # Upscale factor for achieving appropriate longitude dimension
        upscale_factor = 1/(dataset_reader.width / granule_size)
        # Downscale the dataset. Avg scaling to retain integer info
        array = dataset_reader.read(
            out_shape=(
                dataset_reader.count,
                int(dataset_reader.height * upscale_factor),
                int(dataset_reader.width * upscale_factor)
            ),
            resampling=rasterio.enums.Resampling.average
        ).squeeze()
    
    return array

In [19]:
data_folder = Path("../data/raw/hansen")
filename="Hansen_GFC-2018-v1.6_treecover2000_50N_010W.tif"
granule = granule_from_tif(data_folder/filename, granule_size)
granule.shape

(30, 30)

In [20]:
granule

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  5, 12,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  1,  0,  7,  5,  7,  5,  5],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  1,  6,  6,  1,  0,  0,  0,  5, 10, 11, 13,  8],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,
         6, 10, 19, 21, 20, 13,  9, 10, 10,  7,  7,  7, 11, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         7, 16, 13, 17, 21,  9, 17, 16,  7, 12,  6, 11,  9, 12],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         1,  7,  7, 14, 20, 14, 19, 20, 12,  8, 11,  6,  7, 16],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  